System information (for reproducibility):

In [9]:
versioninfo()

Julia Version 1.11.5
Commit 760b2e5b739 (2025-04-14 06:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin24.0.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, apple-m2)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [10]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

Status `~/Documents/github.com/ucla-biostat-257/2025spring/slides/15-linreg/Project.toml`
  [6e4b80f9] BenchmarkTools v1.6.0
  [7522ee7d] SweepOperator v0.3.4
  [37e2e46d] LinearAlgebra v1.11.0


  Activating project at `~/Documents/github.com/ucla-biostat-257/2025spring/slides/15-linreg`


## Comparing methods for linear regression

Methods for solving linear regression $\widehat \beta = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}$:

| Method            | Flops                  | Remarks                 | Software | Stability   |
| :---------------: | :--------------------: | :---------------------: | :------: | :---------: |
| Sweep             | $np^2 + p^3$           | $(X^TX)^{-1}$ available | SAS      | less stable |
| Cholesky          | $np^2 + p^3/3$         |                         |          | less stable |
| QR by Householder | $2np^2 - (2/3)p^3$     |                         | R        | stable      |
| QR by MGS         | $2np^2$                | $Q_1$ available         |          | stable      | 
| QR by SVD         | $4n^2p + 8np^2 + 9p^3$ | $X = UDV^T$             |          | most stable |  

Remarks:

1. When $n \gg p$, sweep and Cholesky are twice faster than QR and need less space.  
2. Sweep and Cholesky are based on the **Gram matrix** $\mathbf{X}^T \mathbf{X}$, which can be dynamically updated with incoming data. They can handle huge $n$, moderate $p$ data sets that cannot fit into memory.  
3. QR methods are more stable and produce numerically more accurate solution.  
4. Although sweep is slower than Cholesky, it yields standard errors and so on.  
5. MGS appears slower than Householder, but it yields $\mathbf{Q}_1$.

> **There is simply no such thing as a universal 'gold standard' when it comes to algorithms.**

## Benchmark

In [11]:
using SweepOperator, BenchmarkTools, LinearAlgebra

linreg_cholesky(y::Vector, X::Matrix) = cholesky!(X'X) \ (X'y)

linreg_qr(y::Vector, X::Matrix) = X \ y

function linreg_sweep(y::Vector, X::Matrix)
    p = size(X, 2)
    xy = [X y]
    tableau = xy'xy
    sweep!(tableau, 1:p)
    return tableau[1:p, end]
end

function linreg_svd(y::Vector, X::Matrix)
    xsvd = svd(X)
    return xsvd.V * ((xsvd.U'y) ./ xsvd.S)
end

linreg_svd (generic function with 1 method)

In [12]:
using Random

Random.seed!(123) # seed

n, p = 10, 3
X = randn(n, p)
y = randn(n)

# check these methods give same answer
@show linreg_cholesky(y, X)
@show linreg_qr(y, X)
@show linreg_sweep(y, X)
@show linreg_svd(y, X);

linreg_cholesky(y, X) = [-0.07196570434574735, -0.13575699455859386, -0.18820199689456507]
linreg_qr(y, X) = [-0.07196570434574738, -0.1357569945585939, -0.18820199689456502]
linreg_sweep(y, X) = [-0.07196570434574734, -0.1357569945585939, -0.188201996894565]
linreg_svd(y, X) = [-0.07196570434574735, -0.13575699455859377, -0.1882019968945651]


In [13]:
n, p = 1000, 300
X = randn(n, p)
y = randn(n)

@benchmark linreg_cholesky(y, X)

BenchmarkTools.Trial: 6327 samples with 1 evaluation per sample.
 Range (min … max):  665.625 μs …   4.052 ms  ┊ GC (min … max): 0.00% … 80.71%
 Time  (median):     750.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   789.258 μs ± 160.343 μs  ┊ GC (mean ± σ):  4.17% ±  9.70%

   ▂ ▂▅██▇▄▃▂▁▁▁▁▁▁                                             ▂
  ███████████████████▅▅▃▃▃▃▁▅▁▁▅▁▆▆████████▇▇▆▆▅▆▅▅▄▆▆▆▆▅▆▆▅▆▆▇ █
  666 μs        Histogram: log(frequency) by time       1.46 ms <

 Memory estimate: 709.20 KiB, allocs estimate: 9.

In [14]:
@benchmark linreg_sweep(y, X)

BenchmarkTools.Trial: 1027 samples with 1 evaluation per sample.
 Range (min … max):  4.493 ms …   8.864 ms  ┊ GC (min … max): 0.00% … 45.70%
 Time  (median):     4.678 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.869 ms ± 413.134 μs  ┊ GC (mean ± σ):  4.02% ±  6.51%

     ▅▇█▇▅▄▃▂▁       ▁▁▁▁    ▁▁ ▁                              
  ▄▁██████████▆▇▆▅▇█▇█████▇█▇█████▅▇▄▇▇█▇▇▇▆▆▆▇▄▇▇▁▇▄▆▆▅▆▄▅▄▆ █
  4.49 ms      Histogram: log(frequency) by time      6.13 ms <

 Memory estimate: 3.04 MiB, allocs estimate: 12.

In [15]:
@benchmark linreg_qr(y, X)

BenchmarkTools.Trial: 360 samples with 1 evaluation per sample.
 Range (min … max):  12.866 ms … 45.692 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.332 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.901 ms ±  2.718 ms  ┊ GC (mean ± σ):  1.43% ± 3.46%

  ▅██▇▆▄▄▃▂ ▃▁▂  ▁                                             
  ██████████████▆█▇▆▄▄▄▆▆▆▄▄▆▁▁▄▁▁▁▆▁▁▁▁▁▁▄▁▄▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▄ ▇
  12.9 ms      Histogram: log(frequency) by time      20.3 ms <

 Memory estimate: 2.49 MiB, allocs estimate: 32.

In [16]:
@benchmark linreg_svd(y, X)

BenchmarkTools.Trial: 191 samples with 1 evaluation per sample.
 Range (min … max):  24.808 ms …  30.799 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     25.986 ms               ┊ GC (median):    1.52%
 Time  (mean ± σ):   26.180 ms ± 851.641 μs  ┊ GC (mean ± σ):  1.92% ± 2.00%

          ▃▄▄▁▂█▆▃   ▂                                          
  ▃▄▄▃▁▆▄▆████████▇▅▇█▇▄▄▇▅▄▅▃▄▃▁▄▄▄▁▁▄▁▃▁▃▁▁▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  24.8 ms         Histogram: frequency by time         29.7 ms <

 Memory estimate: 8.12 MiB, allocs estimate: 27.